In [1]:
import pandas as pd
import glob
import matplotlib.pyplot as plt
import numpy as np

## filepath

In [2]:
#train_data = "../Switchboard-Corpus/swda_data/train_set.txt"
#test_data = "../Switchboard-Corpus/swda_data/test_set.txt"
#val_data = "../Switchboard-Corpus/swda_data/val_set.txt"

## 練習

In [3]:
swda_pass = "../Switchboard-Corpus/swda_data/"
dir_list = ["train", "test", "val"]

In [21]:
len(glob.glob(swda_pass + "test/*"))

19

# 対話ルール設定

In [86]:
user_lists = []

utter_user = []
utter = []
utter_label = []

In [87]:
train_utter_data = []
train_user_data = []
train_label_data = []
test = ""

f = open(swda_pass + "train/2015.txt", "r", encoding='utf-8')
for row in f:
    for i,v in enumerate(row.strip().split("|")):
        
        if i==0 and train_label_data[-1] != v:
            train_label_data.append(v)
    #train_dataset.append(row.strip())
f.close()

In [88]:
utter_user = []
utter = []
utter_label = []

for j in train_dataset:
    for i,v in enumerate(j.split("|")):
        
        if i == 0:
            utter_user.append(v)
        elif i==1:
            utter.append(v)
        else:
            utter_label.append(v)
print("finish")

finish


In [114]:
train_utter_data = []
train_user_data = []
train_label_data = []
text = ""

for i,v in enumerate(utter_user):
    
    if utter_user[i-1] != v:
        #print("実行")
        train_user_data.append(utter_user[i-1])
        train_utter_data.append(text)
        train_label_data.append(utter_label[i-1])
        text = ""
        
    text += utter[i] + " "
    #print(text)

In [115]:
train_utter_data[:10]

["Have you ever gotten one of those calls that is either generated by a computer or somebody going down a list and their either offering a service or they're introducing some new product in the area and normally when they call, you're either in the shower, or you're in the middle of cooking something and you have to stop everything to run to the phone. ",
 "Yes, yes. Is, is that one that you're talking about. ",
 "That was the big one I'm talking about. I work weird hours, and invariably just about the time, I'm going to sleep, the phone tears off the wall. ",
 'Uh-huh, uh-huh. ',
 'And you are trying to crawl out of a half unconscious sleep and answer the phone, you either hear, the as soon as you say hello, you hear the click of the recording coming on, ',
 'Uh-huh. ',
 "or you hear somebody all ready starting, reading off a list of stuff that they've read probably a thousand times that day already. ",
 "That's true, or the ones that are, are generated by a computer. It's just a comp

In [116]:
train_label_data[:10]

['fo_o_fw_"_by_bc', 'qy', 'sd', 'b', 'sd', 'b', 'sd', 'sd', 'sd', 'sd']

In [117]:
train_user_data[:10]

['A', 'B', 'A', 'B', 'A', 'B', 'A', 'B', 'A', 'B']

In [124]:
train_data = tf.data.Dataset.from_tensor_slices((train_utter_data[:-1], train_label_data[1:]))

In [128]:
i,v = next(iter(train_data))
print(i,v)

tf.Tensor(b"Have you ever gotten one of those calls that is either generated by a computer or somebody going down a list and their either offering a service or they're introducing some new product in the area and normally when they call, you're either in the shower, or you're in the middle of cooking something and you have to stop everything to run to the phone. ", shape=(), dtype=string) tf.Tensor(b'qy', shape=(), dtype=string)


In [11]:
labels = set(utter_label)
utter_labels = []
for i in utter_label:
    for j,v in enumerate(labels):
        if i == v:
            utter_labels.append(j)
        

In [8]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [12]:
train_data = tf.data.Dataset.from_tensor_slices((utter, utter_labels))

In [23]:
## ボキャブラリーリスト
vocabulary_set = set()
## トークナイザー
tokenizer = tfds.features.text.Tokenizer()

## 分かち書き
for text_tensor,_ in train_data:
    some_tokens = tokenizer.tokenize(text_tensor.numpy())
    vocabulary_set.update(some_tokens)
    
## ボキャブラリーリスト作成
vocab_size = len(vocabulary_set)

In [34]:
encoder = tfds.features.text.TokenTextEncoder(vocabulary_set)

In [63]:
def encode(token, label):
    token = encoder.encode(token.numpy())
    return token, label

@tf.function
def tf_encoder(utter, label):
    encoded_text, label = tf.py_function(encode,[utter, label],[tf.int64, tf.int32])
    encoded_text.set_shape([None])
    label.set_shape([])
    return encoded_text, label

"""
MAX_LENGTH = 10
def filter_max_length(x,y,max_length=MAX_LENGTH):
    return tf.logical_and(tf.size(x) <= max_length,
                        tf.size(y) <= max_length)
"""

'\nMAX_LENGTH = 10\ndef filter_max_length(x,y,max_length=MAX_LENGTH):\n    return tf.logical_and(tf.size(x) <= max_length,\n                        tf.size(y) <= max_length)\n'

In [46]:
train_data

<TensorSliceDataset shapes: ((), ()), types: (tf.string, tf.int32)>

In [54]:
all_train_data_encode = train_data.map(tf_encoder)

In [55]:
all_train_data_encode

<MapDataset shapes: ((None,), ()), types: (tf.int64, tf.int32)>

In [56]:
for i,v in all_train_data_encode.take(5):
    print(i,v)

tf.Tensor([276], shape=(1,), dtype=int64) tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(
[153 238 267 379 300 354 392 196 342 201 367  50 177 196 177 149 196 326
 285 196  60 369 354 336  95 169], shape=(26,), dtype=int64) tf.Tensor(9, shape=(), dtype=int32)
tf.Tensor(
[275 305 152 288 114 367 392 288 114 306 305  93 263 405  89  77  93  44
 367 139  71 364 354 201 367 392 368 288 114  94 367 392  24 367 392 226
 196  14 312 196 367 392  14 312 357 259], shape=(46,), dtype=int64) tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor([106], shape=(1,), dtype=int64) tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor([198], shape=(1,), dtype=int64) tf.Tensor(0, shape=(), dtype=int32)


In [57]:
all_train_data_encode = all_train_data_encode.filter(filter_max_length)

In [58]:
for i,v in all_train_data_encode.take(5):
    print(i,v)

tf.Tensor([276], shape=(1,), dtype=int64) tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor([106], shape=(1,), dtype=int64) tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor([198], shape=(1,), dtype=int64) tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor([153 387], shape=(2,), dtype=int64) tf.Tensor(5, shape=(), dtype=int32)
tf.Tensor([ 26 228 125 136  77 336  95 169], shape=(8,), dtype=int64) tf.Tensor(6, shape=(), dtype=int32)


In [60]:
BATCH_SIZE = 64
max_len = 40

In [61]:
train_datasets = all_train_data_encode.padded_batch(64, padded_shapes=([max_len], []), drop_remainder=True)

In [75]:
train_batch, train_labels = next(iter(train_datasets))
train_batch.numpy()
train_batch.shape

TensorShape([64, 40])

In [69]:
emb = tf.keras.layers.Embedding(encoder.vocab_size, 64)

In [74]:
result = emb(train_batch)
result.shape

TensorShape([64, 40, 64])

In [7]:
train_dataset = []
f = open(train_data, "r", encoding='utf-8')
for row in f:
    train_dataset.append(row.strip())
f.close()

In [8]:
len(train_dataset)

192390

In [9]:
test_dataset = []
f = open(test_data, "r", encoding='utf-8')
for row in f:
    test_dataset.append(row.strip())
f.close()

In [10]:
len(test_dataset)

4078

In [11]:
val_dataset = []
f = open(val_data, "r", encoding='utf-8')
for row in f:
    val_dataset.append(row.strip())
f.close()

In [12]:
len(val_dataset)

3272

## データの分割

In [13]:
utter_user = []
utter = []
utter_label = []

In [14]:
train_dataset[0]
train_dataset[0].split("|")

['A', 'Okay.', 'fo_o_fw_"_by_bc']

In [15]:
for j in train_dataset:
    for i,v in enumerate(j.split("|")):
        if i == 0:
            utter_user.append(v)
        elif i==1:
            utter.append(v)
        else:
            utter_label.append(v)
print("finish")

finish


# 英語翻訳

### 蛇の目

In [132]:
pip install janome

Note: you may need to restart the kernel to use updated packages.


In [149]:
from janome.tokenizer import Tokenizer

### tensorflow

In [133]:
import tensorflow as tf
import tensorflow_datasets as tfds

### データの取り出し

In [193]:
en_utter = []
jp_utter = []
path = "./ch07/dataset/jpn.txt"

In [194]:
f = open(path, "r", encoding='utf-8')
text = ""
max_length = 0
count = 0
max_length_row = 0

for row in f:
    idx = row.find("_")
    if max_length < idx:
        max_length = idx
        max_length_row = count
    en_utter.append(row[:idx])
    jp_utter.append(row[idx+1:].split("\n")[0])
    count += 1

In [198]:
f.close()

### 配列をランダムに配置させる

In [211]:
import random
def rand_ints_nodup(a, b, k):
    ns = []
    while len(ns) < k:
        n = random.randint(a, b)
        if not n in ns:
            ns.append(n)
    return ns

random_lists = rand_ints_nodup(0, len(en_utter), 5000)

In [213]:
tests_list_jp = []
tests_list_en = []

for i in random_lists:
    tests_list_en.append(en_utter[i])
    tests_list_jp.append(jp_utter[i])

In [214]:
train_list_jp = []
train_list_en = []

for i in range(len(en_utter)):
    if i in random_lists:
        continue
        
    train_list_en.append(en_utter[i])
    train_list_jp.append(jp_utter[i])

In [215]:
len(train_list_en), len(tests_list_en)

(50639, 5000)

### トークナイザ

In [216]:
entokenizer = tfds.features.text.Tokenizer()
jptokenizer = Tokenizer(wakati=True)

In [217]:
inp_tokens, out_tokens = [], []
for i in en_utter:
    inp_tokens += entokenizer.tokenize(i)
for j in jp_utter:
    out_tokens += jptokenizer.tokenize(j)

In [218]:
inp_vocab_lists = list(set(inp_tokens))
out_vocab_lists = list(set(out_tokens))

#### ボキャブサイズ取得

In [219]:
inp_vocab_size = len(inp_vocab_lists)
out_vocab_size = len(out_vocab_lists)

In [220]:
tokenizaer_enutter = tfds.features.text.TokenTextEncoder(inp_vocab_lists)
tokenizaer_jputter = tfds.features.text.TokenTextEncoder(out_vocab_lists)

### データセットの用意

In [222]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_list_en, train_list_jp))

In [223]:
test_dataset = tf.data.Dataset.from_tensor_slices((tests_list_en, tests_list_jp))

### パラメータ設定

In [224]:
BUFFER_SIZE = 50000
BATCH_SIZE = 64
TAKE_SIZE = 5000
MAX_LENGTH = 40
# 埋め込みベクトルの次元
embedding_dim = 256
# RNN ユニットの数
rnn_units = 1024

### エンコーダ

In [225]:
def encode(lang1,lang2):
    lang1 = [tokenizaer_enutter.vocab_size] + tokenizaer_enutter.encode(lang1.numpy()) + [tokenizaer_enutter.vocab_size + 1]
    lang2 = [tokenizaer_jputter.vocab_size] + tokenizaer_jputter.encode(lang2.numpy()) + [tokenizaer_jputter.vocab_size + 1]
    return lang1,lang2

### 文の長さを固定

In [160]:
def filter_max_length(x, y, max_length=MAX_LENGTH):
    return tf.logical_and(tf.size(x) <= max_length,tf.size(y) <= max_length)

In [226]:
def tf_encoder(en, jp):
    return tf.py_function(encode, [en,jp], [tf.int64, tf.int64])

### データの中身確認

In [227]:
train_data = train_dataset.map(tf_encoder)
#train_data = train_data.filter(filter_max_length)
train_data = train_data.cache()
train_data = train_data.shuffle(BUFFER_SIZE).padded_batch(BATCH_SIZE, padded_shapes=([-1], [-1]))
train_data.prefetch(tf.data.experimental.AUTOTUNE)

<PrefetchDataset shapes: ((None, None), (None, None)), types: (tf.int64, tf.int64)>

### モデル作成

In [229]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
        tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [230]:
model = build_model(
    vocab_size=tokenizaer_enutter.vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [231]:
for inp, out in train_data.take(1):
    example_batch_predictions = model(inp)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

InvalidArgumentError: indices[56,0] = 10650 is not in [0, 10650) [Op:ResourceGather]